In [1]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "VTO", source="TCBS")
company2 = Company(symbol = "YBG", source="VCI")
listing = Listing()
fin = finance = Finance(symbol="VTO", source="VCI")



In [12]:
fin.ratio()


Meta                         Chỉ tiêu cơ cấu nguồn vốn                                                                  Chỉ tiêu hiệu quả hoạt động                                                                                                                                Chỉ tiêu khả năng sinh lợi                                                                                                                                 Chỉ tiêu thanh khoản                                                                    Chỉ tiêu định giá                                                                                                                
   ticker yearReport lengthReport (ST+LT borrowings)/Equity Debt/Equity Fixed Asset-To-Equity Owners' Equity/Charter Capital              Asset Turnover Fixed Asset Turnover Days Sales Outstanding Days Inventory Outstanding Days Payable Outstanding  Cash Cycle Inventory Turnover            EBIT Margin (%) Gross Profit Margin (%) Net Profit Margin (%)   ROE (%)   ROIC (%)   ROA (%) EBITDA (Bn. VND) EBIT (Bn. VND) Dividend yield (%)        Current Ratio Cash Ratio Quick Ratio Interest Coverage Financial Leverage Market Capital (Bn. VND) Outstanding Share (Mil. Shares)        P/E       P/B       P/S P/Cash Flow    EPS (VND)    BVPS (VND) EV/EBITDA
0     VTO       2025            2                  0.249357    0.361537              0.387897                       1.393636                    0.715278             2.287768              20.915738                  36.717456                10.951502   68.584696           9.940776                   0.134248                0.232012              0.119146  0.090184   0.090106  0.065468     2.993773e+11    34759981490           0.076271             3.620366   3.015087    3.304420        -12.073143           1.361537             942426658800                        79866666   9.070026  0.846706  0.830168    4.159040   386.266160  13936.359695  0.756521
1     VTO       2025            1                  0.264817    0.311995              0.391305                       1.489349                    0.706117             2.097170              20.762411                  36.526476                12.349619   69.638506           9.992751                   0.073087                0.185557              0.067670  0.089022   0.086237  0.063828     3.000780e+11    20827304507           0.070588             4.401629   3.650636    4.060455         -6.442240           1.311995            1018299991500                        79866666   9.920812  0.856079  0.896772    4.370360   241.447114  14893.485387  1.052357
2     VTO       2024            4                  0.309917    0.380802              0.434376                       1.465204                    0.686393             1.883759              24.310288                  38.894399                15.128947   78.333634           9.384385                   0.112903                0.221719              0.116814  0.095776   0.098736  0.068004     3.087891e+11    33201583424           0.072289             3.619501   2.968770    3.313528         -9.153693           1.380802             994339991700                        79866666   8.971284  0.849711  0.888828    3.642677   430.113426  14652.038273  1.213413
3     VTO       2024            3                  0.344608    0.458407              0.510051                       1.422192                    0.683574             1.773207              22.400156                  39.218585                16.165541   77.784283           9.306812                   0.089763                0.219871              0.065331  0.103547   0.114802  0.072443     3.030542e+11    26683020864           0.065217             2.948421   2.369865    2.693801         -6.832475           1.458407            1102159990800                        79866666   9.318733  0.970333  0.987572    3.978533   243.161798  14221.924846  1.592154
4     VTO       2024            2                  0.384278    0.432663              0.

In [5]:
import pandas as pd
df = pd.DataFrame(fin.income_statement())
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
print(df.head())

  ticker  yearReport  lengthReport  Revenue YoY (%)  Revenue (Bn. VND)  Attribute to parent company (Bn. VND)  Attribute to parent company YoY (%)  Financial Income  Interest Expenses         Sales     Net Sales  Cost of Sales  Gross Profit  Financial Expenses  Selling Expenses  General & Admin Expenses  Operating Profit/Loss  Other income  Net income from associated companies  Other Income/Expenses  Net other income/expenses  Profit before tax  Business income tax - current  Business income tax - deferred  Net Profit For the Year  Attributable to parent company
0    VTO        2025             2        -0.001131       258924368286                            30849790348                             0.042681        8652920238        -2879116310  258924368286  258924368286  -198850912075   60073456211         -3846273143                 0              -25313474721            39566628585      82566932                                     0             -196476547                 -113909615  